### sdfvsfdv

In [ ]:
import osmnx as ox
import networkx as nx 
import ipyleaflet
import sklearn
from ipyleaflet import Map,Marker,Polyline
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Widgets for user input
origin_widget = widgets.Text(
    value='Tehran University, Tehrann',
    description='Origin:',
    layout=widgets.Layout(width='400px')
)
destination_widget = widgets.Text(
    value='Azadi Tower, Tehran',
    description='Destination:',
    layout=widgets.Layout(width='400px')
)
button = widgets.Button(description="Show Route", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        geolocator = Nominatim(user_agent="geoGraphicsApp")
        origin_address = origin_widget.value
        destination_address = destination_widget.value

        origin_location = geolocator.geocode(origin_address + ", Iran",country_codes='ir')
        destination_location = geolocator.geocode(destination_address + ", Iran",country_codes='ir')

        if not origin_location or not destination_location:
            print("Could not geocode one or both addresses.")
            return

        original_coords = [origin_location.latitude, origin_location.longitude]
        destination_coords = [destination_location.latitude, destination_location.longitude]

        # Download the street network for Tehran
        graph = ox.graph_from_place("Tehran, Iran", network_type='drive')

        # Find the nearest nodes to the start and end coordinates
        original_node = ox.distance.nearest_nodes(graph, original_coords[1], original_coords[0])
        destination_node = ox.distance.nearest_nodes(graph, destination_coords[1], destination_coords[0])

        # Compute the shortest path (by length)
        shortest_path = nx.shortest_path(G=graph, source=original_node, target=destination_node, weight='length')

        # Build a list of all points along the path, using edge geometry if available
        path_points = []
        for u, v in zip(shortest_path[:-1], shortest_path[1:]):
            data = graph.get_edge_data(u, v)
            edge = data[0] if 0 in data else list(data.values())[0]
            if 'geometry' in edge:
                xs, ys = edge['geometry'].xy
                segment = list(zip(ys, xs))  # (lat, lon)
                if path_points and path_points[-1] == segment[0]:
                    segment = segment[1:]
                path_points.extend(segment)
            else:
                lat1, lon1 = graph.nodes[u]['y'], graph.nodes[u]['x']
                lat2, lon2 = graph.nodes[v]['y'], graph.nodes[v]['x']
                if not path_points or path_points[-1] != (lat1, lon1):
                    path_points.append((lat1, lon1))
                path_points.append((lat2, lon2))

        midpoint = [(original_coords[0] + destination_coords[0]) / 2, (original_coords[1] + destination_coords[1]) / 2]
        m = Map(center=midpoint, zoom=12)
        m.add_layer(Marker(location=original_coords, draggable=False, title="Start"))
        m.add_layer(Marker(location=destination_coords, draggable=False, title="End"))
        m.add_layer(Polyline(locations=path_points, color="blue", fill=False))
        display(m)

button.on_click(on_button_clicked)

# Display the widgets
display(widgets.VBox([origin_widget, destination_widget, button, output]))